# GraphFlow Advanced(Workflows)
## AutoGen の SocietyOfMindAgent によるネスト型マルチエージェント構成

より複雑なマルチエージェント構成を考えてみます。AutoGen を愛用しているユーザーにとっては、`GraphFlow` と既存の自律性の高い `SelectorGroupChat` や `RoundRobinGroupChat` と組み合わせてみたいと考えるのは自然かと思います。

![](./img/002.png)

今回はこのようなエージェントを実装します。

In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[mcp,openai,azure]"

In [ ]:
from typing import Any, List  
import os  
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
import warnings
warnings.simplefilter('ignore')

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:
server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Number of Tools: {len(tools)}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## チーム編成
`RoundRobinGroupChat` で 2 つの井戸端会議的パターンを定義。例として批判的評価チームと肯定的評価チームに分けてみました。実際は各 `AssistantAgent` には適切なロールとナレッジ、アクションをアタッチします。今回は簡単のため、`TextMessageTermination("critic_agent3")` のようにして会話終了条件を 3 人目のエージェントが発言したら終了するようにしました。

![](./img/001.png)

## エージェント定義

In [ ]:
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMessageTermination  

# Create the final reviewer agent
writer_agent = AssistantAgent(
    "writer_agent",
    model_client=model_client,
    system_message="あなたはプロの小説家です。魅力的な文章を完結なタッチで書くことができます。",
)

# Create the final reviewer agent
critic_agent1 = AssistantAgent(
    "critic_agent1",
    model_client=model_client,
    system_message="作成された文章を批判的な観点から評価し、改善点を提案すること。",
)

# Create the final reviewer agent
critic_agent2 = AssistantAgent(
    "critic_agent2",
    model_client=model_client,
    system_message="あなたはリスクアドバイザーです。作成された文章のアイデア被りやSMS等で炎上しないかどうかを評価し、改善点を提案すること。",
)

# Create the final reviewer agent
critic_agent3 = AssistantAgent(
    "critic_agent3",
    model_client=model_client,
    system_message="あなたは法学のスペシャリストです。作成された文章を法的観点から評価し、改善点を提案すること。",
)

# add agents to array
critic_agents: List[AssistantAgent] = [
    critic_agent1,
    critic_agent2,
    critic_agent3,
]

termination_condition = TextMessageTermination("critic_agent3") #3人目が発言したら終了
critic_team = RoundRobinGroupChat(critic_agents, termination_condition=termination_condition)


# Create the final reviewer agent
progressive_agent1 = AssistantAgent(
    "progressive_agent1",
    model_client=model_client,
    system_message="作成された文章を肯定的な観点から評価し、改善点を提案すること。",
)

# Create the final reviewer agent
progressive_agent2 = AssistantAgent(
    "progressive_agent2",
    model_client=model_client,
    system_message="あなたは先進的なアイデアを持っています。作成された文章を肯定的に評価し、品質を向上させるアイデアを提案すること。",
)

# Create the final reviewer agent
progressive_agent3 = AssistantAgent(
    "progressive_agent3",
    model_client=model_client,
    system_message="あなたは生ける百科事典です。作成された文章を万物を理解している俯瞰的視点から評価し、作品背景をよりリッチで深みのあるものにするために提案すること。",
)

# add agents to array
progressive_agents: List[AssistantAgent] = [
    progressive_agent1,
    progressive_agent2,
    progressive_agent3,
]

termination_condition = TextMessageTermination("progressive_agent3") #3人目が発言したら終了
progressive_team = RoundRobinGroupChat(progressive_agents, termination_condition=termination_condition)

society_of_mind_agent1 = SocietyOfMindAgent("society_of_mind1", team=critic_team, model_client=model_client)
society_of_mind_agent2 = SocietyOfMindAgent("society_of_mind2", team=progressive_team, model_client=model_client)


# Create the final reviewer agent
summary_agent = AssistantAgent(
    "summary_agent",
    model_client=model_client,
    system_message="作成された文章を簡潔なあらすじにして出力すること。",
)


# GraphFlow 定義
並行ファンアウト/ファンインパターンを実装します。

In [ ]:
# Build the workflow graph
builder = DiGraphBuilder()
builder.add_node(writer_agent).add_node(society_of_mind_agent1).add_node(society_of_mind_agent2).add_node(summary_agent)

# Fan-out from writer to editor1 and editor2
builder.add_edge(writer_agent, society_of_mind_agent1)
builder.add_edge(writer_agent, society_of_mind_agent2)

# Fan-in both editors into final reviewer
builder.add_edge(society_of_mind_agent1, summary_agent)
builder.add_edge(society_of_mind_agent2, summary_agent)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)


In [ ]:
flow._participant_names

## グラフの可視化

In [ ]:
#!sudo apt install graphviz -y
#! pip install graphviz

In [ ]:
import graphviz
def visualize_digraph_graphviz(builder):
    dot = graphviz.Digraph(comment="DiGraph Visualization")
    dot.attr(
        rankdir="TB",  # 上→下
        nodesep="0.8",  # ノード間隔
        ranksep="0.5",  # ランク間隔
        edgesep="0.2",  # エッジ間隔
    )

    for name, node in builder.nodes.items():
        # 条件分岐ノード判定: 1つでもcondition付きエッジがあれば分岐ノードとみなす
        is_conditional = any(getattr(edge, "condition", None) for edge in node.edges)
        if is_conditional:
            fill = "pink"  # 条件分岐ノードはオレンジ色
        else:
            fill = "lightgreen" if name == builder._default_start_node else "lightblue"
        dot.node(name, name, style="filled", fillcolor=fill)

    for src, node in builder.nodes.items():
        for edge in node.edges:
            tgt = getattr(edge.target, "name", edge.target)
            cond = edge.condition or ""
            if cond:
                dot.edge(
                    src,
                    tgt,
                    label=cond,
                    style="dashed",
                    minlen="1",
                    labeldistance="2.0",
                    labelangle="0",
                )
            else:
                dot.edge(src, tgt, style="solid")
    return dot

# 使用例
dot_graph = visualize_digraph_graphviz(builder)
dot_graph  # セルの最後でオブジェクトを評価すると自動表示


In [ ]:
with tracer.start_as_current_span("SocietyOfMindAgent") as rollspan: # ルートスパンを作成
    stream = flow.run_stream(task="SEが異世界に転生して無双する異世界転生系短編小説を書いてください。")
    await Console(stream)

## トレース解析から分かる実際のフロー

![](./img/003.png)

### SocietyOfMindAgent
ソサイアティ・オブ・マインド（多重思考）の名前の通り、複数のエージェントが協働してまず内部で議論し、その議論の要旨をもとに最終的な回答だけを LLM に生成させるという役割を担うエージェントです。`SocietyOfMindAgent1` には 3つのエージェントを含む `RoundRobinGroupChat` を内部チームとして含めていますが、チームの議論終了後に `SocietyOfMindAgent` が議論の結果を集約するのがポイントです。
